In [3]:
import pandas as pd
import os
import numpy as np
import pickle
import mne

In [59]:
# this dataset
channels = ["Fp1", "Fp2", "F7", "F3", "Fz", "F4", "F8", "T3", "C3", "Cz", "C4", "T4", "T5", "P3", "Pz", "P4", "T6", "O1", "O2"]

n1 = 65
n2 = 52

labels1 = np.zeros(n1)
labels2 = np.zeros(n2)

patient_dict1 = dict()
for i in range(n1):
    patient_dict1[i] = dict()
    for j in range(len(channels)):
        patient_dict1[i][j] = []

patient_dict2 = dict()
for i in range(n2):
    patient_dict2[i] = dict()
    for j in range(len(channels)):
        patient_dict2[i][j] = []
        

In [9]:
patient_info = pd.read_csv("participants.tsv", delimiter="\t")
print(patient_info.head())
groups = patient_info["Group"]
groups.value_counts()

  participant_id Gender  Age Group  MMSE
0        sub-001      F   57     A    16
1        sub-002      F   78     A    22
2        sub-003      M   70     A    14
3        sub-004      F   67     A    20
4        sub-005      M   70     A    22


Group
A    36
C    29
F    23
Name: count, dtype: int64

In [53]:
gender = patient_info["Gender"]
print(gender)
mapper = {
    'F': 0,
    'M': 1
}
gender = [mapper[g] for g in gender]
print(gender)

0     F
1     F
2     M
3     F
4     M
     ..
83    F
84    M
85    M
86    M
87    M
Name: Gender, Length: 88, dtype: object
[0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1]


In [ ]:
path = './'
file_i = 0
ind1 = 0
ind2 = 0

gender1 = []
gender2 = []

labels1 = []
labels2 = []

for filename in os.listdir(path):
    if 'sub' in filename:
        print(file_i)
        print(ind1, ind2)
        print(filename)
        eeg_file = "{}\eeg\{}_task-eyesclosed_eeg.set".format(filename, filename)
        print(eeg_file)
        data = mne.io.read_raw_eeglab(eeg_file)

        
        for j, c in enumerate(channels):

            signal_data = data.get_data(picks=j)[0]
            signal_data = signal_data
            
            if patient_info["Group"][file_i] == "A":
                patient_dict1[ind1][j] = signal_data
 
            if patient_info["Group"][file_i] == "F":
                patient_dict2[ind2][j] = signal_data

            if patient_info["Group"][file_i] == "C":
                patient_dict1[ind1][j] = signal_data
                patient_dict2[ind2][j] = signal_data
            
        if patient_info["Group"][file_i] == "A": 
            labels1.append(0)
            gender1.append(gender[file_i])
            ind1+=1
        if patient_info["Group"][file_i] == "F": 
            labels2.append(0)
            gender2.append(gender[file_i])
            ind2+=1
        if patient_info["Group"][file_i] == "C":
            labels1.append(1)
            labels2.append(1)
            gender1.append(gender[file_i])
            gender2.append(gender[file_i])
            ind1+=1
            ind2+=1

        file_i+=1

In [46]:
with open('patient_eeg_alz.pickle', 'wb') as fp:
    pickle.dump(patient_dict1, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [47]:
with open('patient_eeg_dem.pickle', 'wb') as fp:
    pickle.dump(patient_dict2, fp)
    print('dictionary saved successfully to file')

dictionary saved successfully to file


In [66]:
labels_df = pd.DataFrame()
labels_df['patient'] = list(range(n1))
labels_df['label1'] = labels1
labels_df['gender'] = gender1
labels_df.to_csv('labels_alz.csv', index=False)
labels_df.head()

,patient,label1,gender
0,0,0,0
1,1,0,0
2,2,0,1
3,3,0,0
4,4,0,1


In [65]:
labels_df = pd.DataFrame()
labels_df['patient'] = list(range(n2))
labels_df['label1'] = labels2
labels_df['gender'] = gender2
labels_df.to_csv('labels_dem.csv', index=False)
labels_df.head()

,patient,label1,gender
0,0,1,1
1,1,1,1
2,2,1,1
3,3,1,1
4,4,1,0
